In [16]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords,twitter_samples
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import re, string, random

In [2]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [3]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [7]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [18]:
#used python shell to download twitter samples
#>>>import nltk
#>>>nltk.download("twitter_samples")

In [17]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

In [19]:
stop_words = stopwords.words('english')

In [20]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

In [22]:
for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [23]:
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [24]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [25]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [26]:
positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

In [27]:
dataset = positive_dataset + negative_dataset

In [28]:
random.shuffle(dataset)

In [29]:
print(len(dataset))

10000


In [30]:
train_data = dataset[:9000]
test_data = dataset[1000:]

In [31]:
classifier = NaiveBayesClassifier.train(train_data)

In [32]:
classify.accuracy(classifier,test_data)

0.9986666666666667

In [33]:
#saving model

In [34]:
import pickle

In [35]:
f=open('model.pickle','wb')
pickle.dump(classifier,f)
f.close()

In [36]:
custom_tweet = "Good morning sir, first of all I would like to thank you for shortlisting me for this interview"

custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(custom_tokens)

['good', 'morning', 'sir', 'first', 'of', 'all', 'i', 'would', 'like', 'to', 'thank', 'you', 'for', 'shortlist', 'me', 'for', 'this', 'interview']


In [38]:
print(custom_tweet,'\n', classifier.classify(dict([token, True] for token in custom_tokens)))

Good morning sir, first of all I would like to thank you for shortlisting me for this interview 
 Positive
